In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader 
from tqdm.notebook import tqdm

In [2]:
vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 200MB/s] 


In [3]:
for param in vgg16.features.parameters():
    param.requires_grad = False

In [4]:
vgg16.classifier[6] = nn.Linear(in_features=4096, out_features=10)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.classifier.parameters(), lr=1e-4)

In [7]:
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # Normalize RGB to [-1, 1]
])

train_dataset = datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/val', transform=transform)


In [9]:
from torch.utils.data import Subset, random_split
import random

# Define the percentage or number of samples you want
num_train_samples = 6000
train_indices = random.sample(range(len(train_dataset)), num_train_samples)
train_subset = Subset(train_dataset, train_indices)

val_size = 1000
train_size = 5000
train_subset, val_subset = random_split(train_subset, [train_size, val_size])

num_test_samples = 1000
test_indices = random.sample(range(len(test_dataset)), num_test_samples)
test_subset = Subset(test_dataset, test_indices)

# Create data loaders
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

In [10]:
epochs = 15
for epoch in range(epochs):
    vgg16.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100. * correct / total

    vgg16.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_acc = 100. * correct / total
    
    print(f"Epoch: {1 + epoch:.2f}%\n")
    print(f"Train Accuracy: {train_acc:.2f}%\n")
    print(f"Validation Accuracy: {val_acc:.2f}%\n")
    print("-" * 50)

Epoch 1/15 - Training:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch 1/15 - Validation:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 1.00%

Train Accuracy: 58.36%

Validation Accuracy: 70.60%

--------------------------------------------------


Epoch 2/15 - Training:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch 2/15 - Validation:   0%|          | 0/16 [00:00<?, ?it/s]

...

Epoch 14/15 - Validation:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 14.00%

Train Accuracy: 100.00%

Validation Accuracy: 73.30%

--------------------------------------------------


Epoch 15/15 - Training:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch 15/15 - Validation:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 15.00%

Train Accuracy: 100.00%

Validation Accuracy: 73.00%

--------------------------------------------------


In [12]:
vgg16.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = 100. * correct / total

In [13]:
test_acc

72.1